In [ ]:
import openpyxl as oxl
from hpath_backend import excel as xl

wbook = oxl.load_workbook('../frontend/static/examples/config.xlsx', data_only=True)

In [ ]:
from hpath_backend.config import Config
from hpath_backend.model import Model

In [ ]:
model = Model(config=Config.from_workbook(wbook, 168*4, 1))

In [ ]:
model.completed_specimens.as_list()

In [ ]:
model.run()

In [ ]:
sp_list = model.completed_specimens.as_list()

In [ ]:
sp_list[1023].data

In [ ]:
from hpath_backend.kpis import Report, tat_by_stage

In [ ]:
report = Report.from_model(model).model_dump()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
np.mean([x.data['processing_end'] - x.data['processing_start'] for x in sp_list if 'init' not in x.name()])

In [ ]:
report['tat_by_stage']

In [ ]:
timestamps = pd.DataFrame.from_dict(
        {sp.name(): sp.data for sp in model.completed_specimens.as_list() if 'init' not in sp.name()},
        orient='index'
    )
timestamps

In [ ]:
timestamps.index = [int(idx.rsplit('.', 1)[1]) for idx in timestamps.index]
timestamps

In [ ]:
stages = [x.rsplit('_end', 1)[0] for x in timestamps.columns if x.endswith('_end')]
stages

In [ ]:
df = pd.concat([timestamps[f'{x}_end'] - timestamps[f'{x}_start']
                    for x in stages], axis='columns')
df.columns = stages
df

In [ ]:
ret = pd.DataFrame({'mean (hours)': df.mean()})
ret

In [ ]:
from hpath_backend import util

In [ ]:
ret.index = [wip.name() for wip in util.dc_values(model.wips)][1:]
ret

In [ ]:
tat_by_stage(model)